不行：

- 直接用openai作为provider
- 使用azure openai也报错

In [1]:
from trulens_eval import Tru

tru = Tru()
tru.reset_database()

🦑 Tru initialized with db url sqlite:///default.sqlite .
🛑 Secret keys may be written to the database. See the `database_redact_keys` option of `Tru` to prevent this.


In [2]:
%%time

from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.core import Settings
from llama_index.llms.openai_like import OpenAILike
from llama_index.embeddings.ollama import OllamaEmbedding

Settings.chunk_size = 128
Settings.chunk_overlap = 16

Settings.llm = OpenAILike(
    model="qwen2", 
    api_base="http://monkey:11434/v1", 
    api_key="ollama",
    is_chat_model=True,
    temperature=0.1,
    request_timeout=60.0
)

Settings.embed_model =OllamaEmbedding(
    model_name="quentinz/bge-large-zh-v1.5",
    base_url="http://monkey:11434",
    ollama_additional_kwargs={"mirostat": 0}, # -mirostat N 使用 Mirostat 采样。
)

documents = SimpleDirectoryReader("data").load_data()
index = VectorStoreIndex.from_documents(documents)

query_engine = index.as_query_engine(similarity_top_k=3)

CPU times: user 410 ms, sys: 28.9 ms, total: 439 ms
Wall time: 8.54 s


In [3]:
response = query_engine.query("北海公园在哪里?")
print(response)

北海公园位于北京市西城区文津街1号。


In [33]:
import os

os.environ["OPENAI_API_KEY"] = "sk-FdNf8kpfMG1yy1pU16D8E82592974693Bf40E2Df39117991"
os.environ["OPENAI_API_BASE"] = "https://api.bianxie.ai/v1"

In [36]:
import numpy as np

from trulens_eval import Feedback
# from trulens_eval.feedback.provider import OpenAI
from trulens_eval.feedback.provider.openai import AzureOpenAI

# Initialize provider class
# provider = OpenAI()
provider =AzureOpenAI(
    # model="gpt-35-turbo",
    deployment_name="test",
    api_key="sk-FdNf8kpfMG1yy1pU16D8E82592974693Bf40E2Df39117991",
    azure_endpoint="https://api.bianxie.ai/v1",
    api_version="2023-07-01-preview",
)


# select context to be used in feedback. the location of context is app specific.
from trulens_eval.app import App

context = App.select_context(query_engine)

# Define a groundedness feedback function
f_groundedness = (
    Feedback(
        provider.groundedness_measure_with_cot_reasons, name="Groundedness"
    )
    .on(context.collect())  # collect context chunks into a list
    .on_output()
)

# Question/answer relevance between overall question and answer.
f_answer_relevance = Feedback(
    provider.relevance_with_cot_reasons, name="Answer Relevance"
).on_input_output()
# Question/statement relevance between question and each context chunk.
f_context_relevance = (
    Feedback(
        provider.context_relevance_with_cot_reasons, name="Context Relevance"
    )
    .on_input()
    .on(context)
    .aggregate(np.mean)
)

✅ In Groundedness, input source will be set to __record__.app.query.rets.source_nodes[:].node.text.collect() .
✅ In Groundedness, input statement will be set to __record__.main_output or `Select.RecordOutput` .
✅ In Answer Relevance, input prompt will be set to __record__.main_input or `Select.RecordInput` .
✅ In Answer Relevance, input response will be set to __record__.main_output or `Select.RecordOutput` .
✅ In Context Relevance, input question will be set to __record__.main_input or `Select.RecordInput` .
✅ In Context Relevance, input context will be set to __record__.app.query.rets.source_nodes[:].node.text .


In [37]:
from trulens_eval import TruLlama

tru_query_engine_recorder = TruLlama(
    query_engine,
    app_id="LlamaIndex_App1",
    feedbacks=[f_groundedness, f_answer_relevance, f_context_relevance],
)

In [38]:
import nltk
nltk.set_proxy('http://myproxy:7890')

# or as context manager
with tru_query_engine_recorder as recording:
    query_engine.query("北海公园在哪里?")

In [39]:
last_record = recording.records[-1]

from trulens_eval.utils.display import get_feedback_result

get_feedback_result(last_record, "Context Relevance")

""


In [40]:
from trulens_eval.guardrails.llama import WithFeedbackFilterNodes

# note: feedback function used for guardrail must only return a score, not also reasons
f_context_relevance_score = Feedback(provider.context_relevance)

filtered_query_engine = WithFeedbackFilterNodes(
    query_engine, feedback=f_context_relevance_score, threshold=0.5
)

In [41]:
tru_recorder = TruLlama(
    filtered_query_engine,
    app_id="LlamaIndex_App1_Filtered",
    feedbacks=[f_answer_relevance, f_context_relevance, f_groundedness],
)

with tru_recorder as recording:
    llm_response = filtered_query_engine.query(
        "北海公园在哪里?"
    )

display(llm_response)

RuntimeError: Endpoint openai request failed 4 time(s): 
	Connection error.
	Connection error.
	Connection error.
	Connection error.